In [22]:
from pymongo import MongoClient
from IPython.display import display, Markdown

# Connect to MongoDB
client = MongoClient("mongodb://localhost:28910/")
db = client["wifi_data_db"]
collection = db["wifi_client_data_2"]

# More robust query that checks if data exists and is an array
pipeline = [
    {
        "$match": {
            "data": {"$exists": True, "$type": "array"}
        }
    },
    {
        "$addFields": {
            "funNetworkCount": {
                "$size": {
                    "$filter": {
                        "input": "$data",
                        "as": "item",
                        "cond": { 
                            "$and": [
                                {"$ifNull": ["$$item.SSID", False]},
                                {"$eq": ["$$item.SSID", "fun_network"]}
                            ]
                        }
                    }
                }
            }
        }
    },
    {
        "$match": {
            "funNetworkCount": 3
        }
    },
    {
        "$count": "totalDocuments"
    }
]

try:
    result = list(collection.aggregate(pipeline))
    count = result[0]["totalDocuments"] if result else 0
    display(Markdown(f"**Number of entries with exactly 3 'fun_network' occurrences:** {count}"))
except Exception as e:
    display(Markdown(f"**Error:** {str(e)}"))

**Number of entries with exactly 3 'fun_network' occurrences:** 43102

In [23]:
import pandas as pd
from IPython.display import display, Markdown

# Set pandas display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

pipeline = [
    {
        "$match": {
            "metadata.pico_ip": {"$exists": True},
            "metadata.button_id": {"$exists": True}
        }
    },
    {
        "$addFields": {
            "y_coordinate": {
                "$toInt": {
                    "$arrayElemAt": [
                        {"$split": ["$metadata.pico_ip", "."]},
                        3
                    ]
                }
            }
        }
    },
    {
        "$group": {
            "_id": {
                "x": "$metadata.button_id",
                "y": "$y_coordinate"
            },
            "count": { "$sum": 1 }
        }
    },
    {
        "$sort": { "_id.x": 1, "_id.y": 1 }
    }
]

try:
    result = list(collection.aggregate(pipeline))
    df = pd.DataFrame([{
        "X (button_id)": item["_id"]["x"],
        "Y (pico_ip last digit)": item["_id"]["y"],
        "Count": item["count"]
    } for item in result])
    
    display(Markdown("**Number of points per X,Y coordinate:**"))
    display(df)
    
    # Reset pandas options to default after display
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.max_colwidth')
except Exception as e:
    display(Markdown(f"**Error:** {str(e)}"))

**Number of points per X,Y coordinate:**

,X (button_id),Y (pico_ip last digit),Count
0,1,30,1182
1,1,31,1182
2,1,32,1182
3,1,33,1182
4,1,34,1182
5,1,35,1182
6,1,36,1182
7,1,37,1182
8,1,38,1182
9,1,39,1182


In [ ]:
import plotly.graph_objects as go
from pymongo import MongoClient
import pandas as pd
from datetime import datetime
from IPython.display import display, Markdown

# Set time interval (UNIX timestamps)
start_time = datetime(2025, 7, 5, 17, 30).timestamp()
end_time = datetime(2025, 7, 5, 19, 00).timestamp()

# Define the custom order for pico IPs
custom_pico_order = [31, 32, 33, 34, 35, 36, 37, 38, 39, 30]
pico_mapping = {val: idx for idx, val in enumerate(custom_pico_order)}

# First get all unique fun_network BSSIDs in the time range
bssid_pipeline = [
    {
        "$match": {
            "timestamp": {"$gte": start_time, "$lte": end_time},
            "data.SSID": "fun_network"
        }
    },
    {"$unwind": "$data"},
    {"$match": {"data.SSID": "fun_network"}},
    {"$group": {"_id": "$data.BSSID"}},
    {"$sort": {"_id": 1}}
]

unique_bssids = [x["_id"] for x in collection.aggregate(bssid_pipeline)]

if not unique_bssids:
    display(Markdown("**No BSSIDs found in the specified time range**"))
else:
    display(Markdown(f"**Found {len(unique_bssids)} fun_network BSSIDs between " 
                   f"{datetime.fromtimestamp(start_time)} and {datetime.fromtimestamp(end_time)}**"))
    
    # Create a figure for each BSSID
    for bssid in unique_bssids:
        # Query for this specific BSSID
        pipeline = [
            {
                "$match": {
                    "timestamp": {"$gte": start_time, "$lte": end_time},
                    "data": {
                        "$elemMatch": {
                            "SSID": "fun_network",
                            "BSSID": bssid
                        }
                    }
                }
            },
            {
                "$addFields": {
                    "y_coordinate": {
                        "$toInt": {
                            "$arrayElemAt": [
                                {"$split": ["$metadata.pico_ip", "."]}, 
                                3
                            ]
                        }
                    },
                    "filtered_data": {
                        "$filter": {
                            "input": "$data",
                            "as": "item",
                            "cond": {
                                "$and": [
                                    {"$eq": ["$$item.SSID", "fun_network"]},
                                    {"$eq": ["$$item.BSSID", bssid]}
                                ]
                            }
                        }
                    }
                }
            },
            {"$unwind": "$filtered_data"},
            {
                "$project": {
                    "x": "$metadata.button_id",
                    "y": "$y_coordinate",
                    "z": "$filtered_data.RSSI",
                    "timestamp": 1,
                    "_id": 0
                }
            }
        ]
        
        data = list(collection.aggregate(pipeline))
        
        if not data:
            display(Markdown(f"No data found for BSSID: {bssid}"))
            continue
            
        df = pd.DataFrame(data)
        
        # Map Y coordinates to custom order positions
        df['y_mapped'] = df['y'].map(pico_mapping)
        
        # Create 3D scatter plot
        fig = go.Figure(data=[go.Scatter3d(
            x=df['x'],
            y=df['y_mapped'],
            z=df['z'],
            mode='markers',
            marker=dict(
                size=5,
                color=df['z'],                # set color to RSSI value
                colorscale='Viridis',         # choose a colorscale
                opacity=0.8,
                colorbar=dict(title='RSSI (dBm)')
            ),
            text=[f"RSSI: {z} dBm<br>Button: {x}<br>Pico: {y}" 
                 for x, y, z in zip(df['x'], df['y'], df['z'])],  # hover text
            hoverinfo='text'
        )])
        
        # Customize layout
        fig.update_layout(
            title=f'WiFi Signal Strength (BSSID: {bssid})<br>'
                  f'Time Range: {datetime.fromtimestamp(start_time)} to {datetime.fromtimestamp(end_time)}<br>'
                  f'{len(df)} measurements',
            scene=dict(
                xaxis_title='X Coordinate (button_id)',
                yaxis_title='Y Coordinate (pico_ip)',
                zaxis_title='RSSI (dBm)',
                yaxis=dict(
                    tickvals=list(range(len(custom_pico_order))),
                    ticktext=[str(x) for x in custom_pico_order]
                )
            ),
            width=1000,
            height=800,
            margin=dict(r=20, l=10, b=10, t=80)
        )
        
        # Show the figure
        fig.show()

**Found 3 fun_network BSSIDs between 2025-07-05 17:30:00 and 2025-07-05 19:00:00**